# Solving Continuous Algebraic Riccati Equations with Python

We want to solve the equation $$A^T P + PA - PBR^{-1}B^TP + Q - \rho P = 0 \tag{$\star$}$$ for the matrix $P$, where $$ \rho = 0.1, \quad A = \begin{bmatrix} 0 & 1 \\ -2 & -3\end{bmatrix}, \quad B = \begin{bmatrix} 0 \\ 1\end{bmatrix}, \quad G = \begin{bmatrix} 0.1 & 0 \\ 0 & 0.1\end{bmatrix}, \quad Q = \begin{bmatrix} 1 & 0 \\ 0 & 1\end{bmatrix}, \quad R = [1]$$

Import the following Python modules:

In [1]:
import numpy as np
from scipy.linalg import solve_continuous_are

Initialize the parameters:

In [17]:
rho = 0.1
A = np.array([[0, 1], [-2, -3]])
B = np.array([[0, 1]]).T
G = 0.1*np.eye(2) 
Q = np.eye(2)
R = np.eye(1)

print(A)
print(B)
print(G)
print(Q)
print(R)

[[ 0  1]
 [-2 -3]]
[[0]
 [1]]
[[0.1 0. ]
 [0.  0.1]]
[[1. 0.]
 [0. 1.]]
[[1.]]


The function `solve_continuous_are(a, b, q, r, e = None, s = None, balanced = True)` solves the continuous-time algebraic Riccati equation $$A^TX + XA - XBR^{-1}B^T X + Q = 0 \tag{$\star \star$}$$ 

Here, $H$ means the conjugate transpose (which in the case of real matrices, is just transpose). 

Since the equation $(\star \star)$ we can solve with `solve_continuous_are` is in a different form than $(\star)$, we have to modify $A \to A - \frac{\rho}{2} A$ in $(\star \star)$.

In [18]:
A_mod = A - (rho/2)*A

Now, we can call the function.

In [19]:
P = solve_continuous_are(A, B, Q, R)
print(P)

[[1.23606798 0.23606798]
 [0.23606798 0.23606798]]


The solution $P$ is $$\begin{bmatrix} 1.23606798 & 0.23606798 \\ 0.23606798 &0.23606798\end{bmatrix}$$

Now, $V(x) = x^TPx + c$ where $c = \mathrm{tr}(GG^T P)/\rho$. We compute $c$:

In [26]:
c = np.trace(G*G.T*P)/rho
print(c)

0.002000000000000001


So, $$V(x, y) = [x \ y] \begin{bmatrix} 1.23606798 & 0.23606798 \\ 0.23606798 &0.23606798\end{bmatrix} \begin{bmatrix} x \\ y\end{bmatrix} + 0.002 = 1.23606798 x^2 + 0.47213596xy + 0.23606798y^2 + 0.002.$$

We can write $V$ as a Python function:

In [31]:
def V(x, y):
    return 1.23606798*x**2 + 0.47213596*x*y + 0.23606798*y**2 + 0.002

The optimal control $u$ is given by $u(x) = -R^{-1} B^T P x$. We first compute $R^{-1}B^T P$:

In [33]:
print(np.linalg.inv(R)*B.T*P)

[[0.   0.  ]
 [0.   0.01]]


So, the optimal control is $$u([x \ y]) = - \begin{bmatrix} 0 & 0 \\ 0 & 0.01\end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} = -0.01y.$$